In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('./drive/MyDrive/nixtlats')
print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/nixtlats


In [ ]:
#default_exp models.nbeats.nbeats_model_ensemble

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# !pip install pytorch-lightning
# !pip install torchinfo
# !pip install fastcore
# !pip install s3fs
# !pip install patool

In [ ]:
#export
from dataclasses import dataclass

import pytorch_lightning as pl
from itertools import product
from pathlib import Path
from typing import Callable, Dict, Iterable, Union, List

import pandas as pd
from tqdm import tqdm
import pylab as plt
from pylab import rcParams

from nixtla.models.nbeats.nbeats_model import NBEATS
from nixtla.data.datasets.m4 import M4Info, M4, M4Evaluation
from nixtla.data.tsdataset import TimeSeriesDataset
from nixtla.data.tsloader import TimeSeriesLoader
from nixtla.experiments.utils import create_datasets, get_mask_dfs

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#export
def _parameter_grid(grid):
    specs_list = list(product(*list(grid.values())))
    model_specs_df = pd.DataFrame(specs_list, columns=list(grid.keys()))
    
    return model_specs_df

In [ ]:
#export
common_grid = {}

# Architecture parameters
common_grid['activation'] = ['ReLU'] # Oreshkin
common_grid['n_x'] = [0] # No exogenous variables
common_grid['n_s'] = [0] # No static variables
common_grid['n_x_hidden'] = [0] # No exogenous variables
common_grid['n_s_hidden'] = [0] # No static variables
common_grid['stack_types'] = [['trend', 'seasonality']] # NBEATS-I original architecture
common_grid['n_blocks'] = [[3, 3]] # Trend blocks, Seasonal blocks - Oreshkin
common_grid['n_layers'] = [[4, 4]] # Trend-block layers, Seasonal-block - Oreshkin
common_grid['shared_weights'] = [True] # Oreshkin
common_grid['n_harmonics'] = [1] # Oreshkin
common_grid['n_polynomials'] = [2] # Trend polynomial degree
common_grid['n_theta_hidden'] = [[common_grid['n_layers'][0][0] * [256],
                                  common_grid['n_layers'][0][1] * [2048]]] # Oreshkin
common_grid['initialization'] = ['lecun_normal'] # Arbitrary

# Optimization parameters
common_grid['learning_rate'] = [0.001] # Oreshkin
common_grid['lr_decay'] = [0] # No lr_decay in the original implementation
common_grid['lr_decay_step_size'] = [1_000] # No lr_decay in the original implementation
common_grid['loss_train'] = ['MAPE', 'SMAPE'] # MASE not available. Oreshkin
common_grid['loss_hypar'] = [0.5] # ???
# common_grid['loss_valid'] = common_grid['loss_train'] # Oreskin NOT INCLUDED TO AVOID DUPLICITY
common_grid['dropout_prob_theta'] = [0] # No dropout in the original implementation
common_grid['weight_decay'] = [0] # # No weight_decay in the original implementation
common_grid['batch_size'] = [1024] # Oreshkin
common_grid['batch_normalization'] = [False] # No batch_normalization in the original implementation

common_grid['max_steps'] = [1_000] # Oreshkin
common_grid['random_seed'] = list(range(1)) # Change to range(10). Oreshkin

# Data Parameters
common_grid['complete_inputs'] = [True] # ???
common_grid['mode'] = ['simple'] # Step = 1 window
lookbacks = list(range(2, 3)) # Change to range(2, 8). Oreshkin

In [ ]:
#export
@dataclass
class Yearly:
    group = M4Info['Yearly']

    grid_freq = {}
    grid_freq['max_epochs'] = [1] # In combination with max_n_steps, it trains max_n_epochs*max_n_steps
    grid_freq['n_time_in'] = [M4Info['Yearly'].horizon * i for i in lookbacks]
    grid_freq['n_time_out'] = [group.horizon]
    grid_freq['idx_to_sample_freq'] = [1] # ???
    grid_freq['val_idx_to_sample_freq'] = [1] # ???
    grid_freq['frequency'] = ['Q'] # ???
    grid_freq['seasonality'] = [4] # ???
    grid_freq['l_h'] = [1.5]

    grid = {**common_grid,
            **grid_freq}

In [ ]:
#export
@dataclass
class Quarterly:
    group = M4Info['Quarterly']

    grid_freq = {}
    grid_freq['max_epochs'] = [1] # In combination with max_n_steps, it trains max_n_epochs*max_n_steps
    grid_freq['n_time_in'] = [M4Info['Quarterly'].horizon * i for i in lookbacks]
    grid_freq['n_time_out'] = [group.horizon]
    grid_freq['idx_to_sample_freq'] = [1] # ???
    grid_freq['val_idx_to_sample_freq'] = [1] # ???
    grid_freq['frequency'] = ['Y'] # ???
    grid_freq['seasonality'] = [1] # ???
    grid_freq['l_h'] = [1.5]

    grid = {**common_grid,
            **grid_freq}

In [ ]:
print(f'Yearly grid (# of different model configurations={len(_parameter_grid(Yearly.grid))}):')
print(75*'=')
print(pd.Series(Yearly.grid))
print(75*'=')
print()
print(f'Quarterly grid (# of different model configurations={len(_parameter_grid(Quarterly.grid))}):')
print(75*'=')
print(pd.Series(Quarterly.grid))
print(75*'=')

Yearly grid (# of different model configurations=2):
activation                                                           [ReLU]
n_x                                                                     [0]
n_s                                                                     [0]
n_x_hidden                                                              [0]
n_s_hidden                                                              [0]
stack_types                                          [[trend, seasonality]]
n_blocks                                                           [[3, 3]]
n_layers                                                           [[4, 4]]
shared_weights                                                       [True]
n_harmonics                                                             [1]
n_polynomials                                                           [2]
n_theta_hidden            [[[256, 256, 256, 256], [2048, 2048, 2048, 204...
initialization                     

In [ ]:
# freq = Yearly
# Y_df, _, S_df = M4.load(directory='data', group=freq.group.name)

# freq_grid = _parameter_grid(freq.grid)
# row_ensemble = freq_grid.iloc[0]
# hparams = row_ensemble.to_dict()

# train_dataset = TimeSeriesDataset(Y_df=Y_df, S_df=S_df,
#                                 ds_in_test=freq.group.horizon,
#                                 mode=hparams['mode'],
#                                 window_sampling_limit=hparams['window_sampling_limit'], # To limit backprop time
#                                 input_size=hparams['n_time_in'],
#                                 output_size=hparams['n_time_out'],
#                                 idx_to_sample_freq=hparams['idx_to_sample_freq'],
#                                 complete_inputs=hparams['complete_inputs'], 
#                                 skip_nonsamplable=True)

# train_loader = TimeSeriesLoader(dataset=train_dataset,
#                                             batch_size=int(hparams['batch_size']),
#                                             eq_batch_size=True,
#                                             num_workers=4,
#                                             shuffle=False)


In [ ]:
# batch = next(iter(train_loader))
# print(batch['S'].shape)
# print(batch['Y'].shape)
# print(batch['X'].shape)
# print(batch['available_mask'].shape)

# display(pd.Series(hparams))

In [ ]:
# from torchinfo import summary

# model = NBEATS(n_time_in=int(hparams['n_time_in']),
#                                n_time_out=int(hparams['n_time_out']),
#                                n_x=hparams['n_x'],
#                                n_s=hparams['n_s'],
#                                n_s_hidden=int(hparams['n_s_hidden']),
#                                n_x_hidden=int(hparams['n_x_hidden']),
#                                shared_weights=hparams['shared_weights'],
#                                initialization=hparams['initialization'],
#                                activation=hparams['activation'],
#                                stack_types=hparams['stack_types'],
#                                n_blocks=hparams['n_blocks'],
#                                n_layers=hparams['n_layers'],
#                                n_theta_hidden=hparams['n_theta_hidden'],
#                                n_harmonics=int(hparams['n_harmonics']),
#                                n_polynomials=int(hparams['n_polynomials']),
#                                batch_normalization = hparams['batch_normalization'],
#                                dropout_prob_theta=hparams['dropout_prob_theta'],
#                                learning_rate=float(hparams['learning_rate']),
#                                lr_decay=float(hparams['lr_decay']),
#                                lr_decay_step_size=float(hparams['lr_decay_step_size']),
#                                weight_decay=hparams['weight_decay'],
#                                loss_train=hparams['loss_train'],
#                                loss_hypar=float(hparams['loss_hypar']),
#                                loss_valid='SMAPE',
#                                frequency=hparams['frequency'],
#                                seasonality=int(hparams['seasonality']),
#                                random_seed=int(hparams['random_seed']))

# print(model)
# # summary(model.model, input_size=[(1024, 1), (1024, 18), (1024, 1, 18), (1024, 18)])

In [ ]:
#export
class EnsembleNBEATS:
    # TODO: Update test TimeSeriesDataset instantiation wit parameter last_samplable_window
    def __init__(self,
                 frequencies: List[type],
                 loader: callable,
                 num_workers: int):
        self.frequencies = frequencies
        self.loader = loader
        
        self.num_workers = num_workers

    def fit(self):
        results = {}

        for freq in self.frequencies:
            print(f'\n{freq.group.name}')
            Y_df, _, S_df = M4.load(directory='data', group=freq.group.name)
            freq_grid = _parameter_grid(freq.grid)

            forecasts = []

            for idx_ensemble, row_ensemble in tqdm(freq_grid.iterrows(), position=0, leave=True):
                hparams = row_ensemble.to_dict()
                print(hparams['n_time_in'] + \
                                        int(hparams['n_time_out'] * hparams['l_h']))
                train_dataset = \
                    TimeSeriesDataset(Y_df=Y_df, S_df=S_df,
                                      ds_in_test=freq.group.horizon,
                                      mode=hparams['mode'],
                                      window_sampling_limit=\
                                        hparams['n_time_in'] + \
                                        int(hparams['n_time_out'] * hparams['l_h']), # To limit backprop time 
                                      input_size=hparams['n_time_in'],
                                      output_size=hparams['n_time_out'],
                                      idx_to_sample_freq=hparams['idx_to_sample_freq'],
                                      complete_inputs=hparams['complete_inputs'], 
                                      skip_nonsamplable=True)
                    
                train_loader = TimeSeriesLoader(dataset=train_dataset,
                                                batch_size=hparams['batch_size'],
                                                eq_batch_size=True,
                                                num_workers=self.num_workers,
                                                shuffle=False)
                    
                test_dataset = \
                    TimeSeriesDataset(Y_df=Y_df, S_df=S_df,
                                      ds_in_test=0,
                                    #   mode=hparams['mode'],
                                      mode='full',
                                      window_sampling_limit=\
                                        hparams['n_time_out'] + hparams['n_time_in'], # To limit backprop time 
                                      input_size=hparams['n_time_in'],
                                      output_size=hparams['n_time_out'],
                                    #   idx_to_sample_freq=hparams['idx_to_sample_freq'],
                                      idx_to_sample_freq=1,
                                    #   complete_inputs=hparams['complete_inputs'],
                                      complete_inputs=True, 
                                      complete_outputs=True,
                                    #   last_samplable_window=True,
                                      skip_nonsamplable=False)
                    
                test_loader = TimeSeriesLoader(dataset=test_dataset,
                                               batch_size=1024,
                                               eq_batch_size=False,
                                               num_workers=self.num_workers,
                                               shuffle=False)

                model = NBEATS(n_time_in=int(hparams['n_time_in']),
                               n_time_out=int(hparams['n_time_out']),
                               n_x=hparams['n_x'],
                               n_s=hparams['n_s'],
                               n_s_hidden=int(hparams['n_s_hidden']),
                               n_x_hidden=int(hparams['n_x_hidden']),
                               shared_weights=hparams['shared_weights'],
                               initialization=hparams['initialization'],
                               activation=hparams['activation'],
                               stack_types=hparams['stack_types'],
                               n_blocks=hparams['n_blocks'],
                               n_layers=hparams['n_layers'],
                               n_theta_hidden=hparams['n_theta_hidden'],
                               n_harmonics=int(hparams['n_harmonics']),
                               n_polynomials=int(hparams['n_polynomials']),
                               batch_normalization = hparams['batch_normalization'],
                               dropout_prob_theta=hparams['dropout_prob_theta'],
                               learning_rate=float(hparams['learning_rate']),
                               lr_decay=float(hparams['lr_decay']),
                               lr_decay_step_size=float(hparams['lr_decay_step_size']),
                               weight_decay=hparams['weight_decay'],
                               loss_train=hparams['loss_train'],
                               loss_hypar=float(hparams['loss_hypar']),
                               loss_valid='SMAPE',
                               frequency=hparams['frequency'],
                               seasonality=int(hparams['seasonality']),
                               random_seed=int(hparams['random_seed']))
                
                print(f'\nModel distinctive attributes: loss: {model.loss_train}, random_seeds: {model.random_seed}')
                
                trainer = pl.Trainer(max_epochs=hparams['max_epochs'], 
                                     max_steps=hparams['max_steps'],
                                     gradient_clip_val=0,
                                     progress_bar_refresh_rate=1, 
                                     log_every_n_steps=100, 
                                    #  val_check_interval=0,
                                    #  check_val_every_n_epoch=hparams['max_epochs'],
                                     gpus=-1,
                                     auto_select_gpus=True)
                
                trainer.fit(model, train_loader)

                outputs = trainer.predict(model, test_loader)

                forecasts.append(outputs)             

            results[freq.group.name] = forecasts
            
        return results


In [ ]:
ensemble = EnsembleNBEATS(frequencies=[Yearly, Quarterly],
                          loader=TimeSeriesLoader,
                          num_workers=4)
forecasts = ensemble.fit()

0it [00:00, ?it/s]


Yearly
21


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | _NBEATS | 12.8 M
----------------------------------
12.8 M    Trainable params
162       Non-trainable params
12.8 M    Total params
51.371    Total estimated model params size (MB)



Model distinctive attributes: loss: MAPE, random_seeds: 0


RecursionError: ignored

In [ ]:
y_hat_yearly = forecasts['Yearly']
print(y_hat_yearly[0].shape)

torch.Size([6951, 6])
